This notebook covers my initial exploration of the video game dataset I'll be using to view trends in video game data. I will not be cleansing or changing the file, simply exploring the raw file to determine what steps will be taken next to clean the file. 


Dataset: [Kaggle Video Game Dataset](https://www.kaggle.com/datasets/thedevastator/global-video-game-sales-ratings?resource=download)

### Definitions ###

- **Name**	The name of the video game. (String)
- **Platform**	The platform the game was released on. (String)
- **Year_of_Release**	The year the game was released. (Integer)
- **Genre**	The genre of the game. (String)
- **Publisher**	The publisher of the game. (String)
- **NA_Sales**	The sales of the game in North America. (Float)
- **EU_Sales**	The sales of the game in Europe. (Float)
- **JP_Sales**	The sales of the game in Japan. (Float)
- **Other_Sales**	The sales of the game in other regions. (Float)
- **Global_Sales**	The total sales of the game across all regions. (Float)
- **Critic_Score**	The score given to the game by critics. (Float)
- **Critic_Count**	The number of critics who reviewed the game. (Integer)
- **User_Score**	The score given to the game by users. (Float)
- **User_Count**	The number of users who reviewed the game. (Integer)
- **Developer**	The developer of the game. (String)
- **Rating**	The rating of the game. (String)

---

In [51]:
import pandas as pd

In [52]:
#creating initial dataframe from csv
vg_df = pd.read_csv('data/raw-data.csv')

Some questions I would like to potentially explore are:

1. Which video game genre is the most popular worldwide? What about region to region?
2. What does genre popularity look like over time for video games? By count of genre and/or by sales normalized by total
3. What publisher has the most global sales? Is that the same region to region?

In [53]:
vg_df.columns

Index(['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score',
       'Critic_Count', 'User_Score', 'User_Count', 'Developer', 'Rating'],
      dtype='object')

### Cleaning Step 1

Knowing what I would like to explore for my project, my initial step should be to remove any unneeded columns. 

**Columns to Keep:** Name, Platform, Year_of_Release, Genre, NA_Sales, EU_Sales, JP_Sales, Other_Sales, Global Sales, Publisher

**Columns to Remove:** Critic_Score, User_Score, User_Count, Developer, Rating

In [11]:
vg_df.shape

(16719, 16)

In [12]:
vg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16717 non-null  object 
 1   Platform         16719 non-null  object 
 2   Year_of_Release  16450 non-null  float64
 3   Genre            16717 non-null  object 
 4   Publisher        16665 non-null  object 
 5   NA_Sales         16719 non-null  float64
 6   EU_Sales         16719 non-null  float64
 7   JP_Sales         16719 non-null  float64
 8   Other_Sales      16719 non-null  float64
 9   Global_Sales     16719 non-null  float64
 10  Critic_Score     8137 non-null   float64
 11  Critic_Count     8137 non-null   float64
 12  User_Score       10015 non-null  object 
 13  User_Count       7590 non-null   float64
 14  Developer        10096 non-null  object 
 15  Rating           9950 non-null   object 
dtypes: float64(9), object(7)
memory usage: 2.0+ MB


In [13]:
#looking for null values in dataset

vg_df.isnull().sum().sort_values(ascending=False)

User_Count         9129
Critic_Score       8582
Critic_Count       8582
Rating             6769
User_Score         6704
Developer          6623
Year_of_Release     269
Publisher            54
Name                  2
Genre                 2
Platform              0
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
dtype: int64

I have no missing sales data, which is great for my questions I would like to ask. I will need to assess the null values in the Year_of_Release column when using that data. 

I think to make my analysis more relevant for myself, I will isolate some questions to look at just NA_Sales data after anwsering my initial question of video game popularity by sales. 

In [16]:
vg_df['Year_of_Release'].head()

0    2006.0
1    1985.0
2    2008.0
3    2009.0
4    1996.0
Name: Year_of_Release, dtype: float64

**Cleanup Note**: I'll want to tweak the formatting on the year to drop the decimal

## Question 1

Which video game genre is the most popular worldwide, by sales and by count games? What about region to region?


To know this, I will need to first group the data set by genre and global sales. The genres that have the most sales are the the ones that are most popular. I think for ease of reporting, I will take the top five or ten, but I'd like to see how many unique genres there are. I'll explore that below. 

In [18]:
vg_df['Genre'].unique()

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', nan], dtype=object)

In [20]:
vg_df['Genre'].value_counts()

Action          3370
Sports          2348
Misc            1750
Role-Playing    1500
Shooter         1323
Adventure       1303
Racing          1249
Platform         888
Simulation       874
Fighting         849
Strategy         683
Puzzle           580
Name: Genre, dtype: int64

There are not as many genres as I would expect, so we can report on all of them with our analysis. 

Steps Needed for this data set: 
1. Because genre has 2 null values, I'll want to remove those from my data set
2. Group that cleaned data set and include count of genre & global sales

For the follow-up question, I'll need to 
1. group the four breakout of sales by genre as well and include a count

In [23]:
#exploring how this data looks 
vg_df[['Name','Genre','NA_Sales','EU_Sales','JP_Sales', 'Other_Sales']].head()

,Name,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Wii Sports,Sports,41.36,28.96,3.77,8.45
1,Super Mario Bros.,Platform,29.08,3.58,6.81,0.77
2,Mario Kart Wii,Racing,15.68,12.76,3.79,3.29
3,Wii Sports Resort,Sports,15.61,10.93,3.28,2.95
4,Pokemon Red/Pokemon Blue,Role-Playing,11.27,8.89,10.22,1.00


## Question 2: What does genre popularity look like over time for video games? 

For this question, I will want to look at the category Genre, and the Year_of_Release. I'll likely need to group the years of release into decades or groupings in order to view them in a better way. I'll take a look at the data to see what might make sense. 

In [54]:
vg_df["Year_of_Release"].min()

1980.0

In [55]:
vg_df["Year_of_Release"].max()

2020.0

In [58]:
vg_df['Year_of_Release'].value_counts()

2008.0    1427
2009.0    1426
2010.0    1255
2007.0    1197
2011.0    1136
2006.0    1006
2005.0     939
2002.0     829
2003.0     775
2004.0     762
2012.0     653
2015.0     606
2014.0     581
2013.0     544
2016.0     502
2001.0     482
1998.0     379
2000.0     350
1999.0     338
1997.0     289
1996.0     263
1995.0     219
1994.0     121
1993.0      62
1981.0      46
1992.0      43
1991.0      41
1982.0      36
1986.0      21
1989.0      17
1983.0      17
1990.0      16
1987.0      16
1988.0      15
1985.0      14
1984.0      14
1980.0       9
2017.0       3
2020.0       1
Name: Year_of_Release, dtype: int64

I think it will make sense to group them by decade. 

In [62]:
#dropping NA values before trying to group them 
vg_df.dropna(subset=['Year_of_Release'], inplace=True)

labels = ["1980's", "1990's", "2000's", "2010's", "2020's"]
bins = [1980, 1990, 2000, 2010, 2020, 2030]
vg_df['Decade'] = pd.cut(vg_df['Year_of_Release'], bins=bins, labels=labels, right=False)
vg_df['Decade'].value_counts()

2000's    9193
2010's    5280
1990's    1771
1980's     205
2020's       1
Name: Decade, dtype: int64

Now that I have my decades, I need to look more granular at genres by decade

In [71]:
vg_df.groupby(['Decade', 'Genre'])['Name'].count()

Decade  Genre       
1980's  Action            66
        Adventure          2
        Fighting           4
        Misc               8
        Platform          33
        Puzzle            19
        Racing             8
        Role-Playing       9
        Shooter           30
        Simulation         3
        Sports            23
        Strategy           0
1990's  Action           162
        Adventure         97
        Fighting         193
        Misc             116
        Platform         125
        Puzzle            71
        Racing           183
        Role-Playing     172
        Shooter          137
        Simulation        86
        Sports           304
        Strategy         123
2000's  Action          1584
        Adventure        631
        Fighting         441
        Misc            1021
        Platform         567
        Puzzle           365
        Racing           798
        Role-Playing     732
        Shooter          719
        Simulation    

I'll have to think of a good way to vizualize this data. This gets me far enough along I feel comfortable anwsering my second question. I'll probably want to remove the "2020's" category since there is only one data point. 

## What publisher has the most global sales? Is that the same region to region?

For this question I'll need publisher, and the sales numbers totals and for each region. 

In [70]:
pub_sales = vg_df.groupby(['Publisher'])['Global_Sales'].sum().sort_values(ascending=True)
print(pub_sales)

Publisher
Naxat Soft                        0.01
Commseed                          0.01
Inti Creates                      0.01
Michaelsoft                       0.01
UIG Entertainment                 0.01
                                ...   
Ubisoft                         470.45
Sony Computer Entertainment     606.27
Activision                      725.14
Electronic Arts                1100.05
Nintendo                       1786.72
Name: Global_Sales, Length: 579, dtype: float64


In [75]:
sales_by_publisher = vg_df.groupby('Publisher')[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']].sum()
print(sales_by_publisher)

                              NA_Sales  EU_Sales  JP_Sales  Other_Sales  \
Publisher                                                                 
10TACLE Studios                   0.07      0.04      0.00         0.00   
1C Company                        0.01      0.07      0.00         0.01   
20th Century Fox Video Games      1.82      0.10      0.00         0.01   
2D Boy                            0.00      0.03      0.00         0.01   
3DO                               6.48      3.04      0.00         0.63   
...                                ...       ...       ...          ...   
id Software                       0.02      0.00      0.00         0.00   
imageepoch Inc.                   0.00      0.00      0.04         0.00   
inXile Entertainment              0.03      0.06      0.00         0.01   
mixi, Inc                         0.00      0.00      0.87         0.00   
responDESIGN                      0.09      0.04      0.00         0.01   

                        

In [79]:
na_sorted = sales_by_publisher.sort_values('NA_Sales', ascending=False)
top_5_na = na_sorted['NA_Sales'].head(5)
print(top_5_na)

Publisher
Nintendo                       815.86
Electronic Arts                588.65
Activision                     428.92
Sony Computer Entertainment    266.17
Ubisoft                        252.12
Name: NA_Sales, dtype: float64


In [82]:
eu_sorted = sales_by_publisher.sort_values('EU_Sales', ascending=False)
top_5_eu = na_sorted['EU_Sales'].head(5)
print(top_5_eu)

jp_sorted = sales_by_publisher.sort_values('JP_Sales', ascending=False)
top_5_jp = jp_sorted['JP_Sales'].head(5)
print(top_5_jp)

other_sorted = sales_by_publisher.sort_values('Other_Sales', ascending=False)
top_5_other = other_sorted['Other_Sales'].head(5)
print(top_5_other)

global_sorted = sales_by_publisher.sort_values('Global_Sales', ascending=False)
top_5_global = na_sorted['Global_Sales'].head(5)
print(top_5_global)

SyntaxError: unexpected character after line continuation character (1707368982.py, line 3)

In [83]:
jp_sorted = sales_by_publisher.sort_values('JP_Sales', ascending=False)
top_5_jp = jp_sorted['JP_Sales'].head(5)
print(top_5_jp)

Publisher
Nintendo                        457.74
Namco Bandai Games              127.62
Konami Digital Entertainment     91.03
Sony Computer Entertainment      74.15
Capcom                           67.73
Name: JP_Sales, dtype: float64


In [84]:
other_sorted = sales_by_publisher.sort_values('Other_Sales', ascending=False)
top_5_other = other_sorted['Other_Sales'].head(5)
print(top_5_other)

Publisher
Electronic Arts                126.82
Nintendo                        94.55
Sony Computer Entertainment     79.62
Activision                      75.27
Take-Two Interactive            55.68
Name: Other_Sales, dtype: float64


In [89]:
global_sorted = sales_by_publisher.sort_values('Global_Sales', ascending=False)
top_5_global = global_sorted['Global_Sales'].head(5)
print(top_5_global)

Publisher
Nintendo                       1786.72
Electronic Arts                1100.05
Activision                      725.14
Sony Computer Entertainment     606.27
Ubisoft                         470.45
Name: Global_Sales, dtype: float64


Maybe I'll combine them all for easier data viewing

In [108]:
#combining all of the new series into a new dataframe for better visualizing 
pub_comp_df = pd.concat([top_5_global, top_5_other,top_5_jp, top_5_na ], axis=1).fillna('NA')
print(pub_comp_df)

                             Global_Sales Other_Sales JP_Sales NA_Sales
Publisher                                                              
Nintendo                          1786.72       94.55   457.74   815.86
Electronic Arts                   1100.05      126.82       NA   588.65
Activision                         725.14       75.27       NA   428.92
Sony Computer Entertainment        606.27       79.62    74.15   266.17
Ubisoft                            470.45          NA       NA   252.12
Take-Two Interactive                   NA       55.68       NA       NA
Namco Bandai Games                     NA          NA   127.62       NA
Konami Digital Entertainment           NA          NA    91.03       NA
Capcom                                 NA          NA    67.73       NA


I think this gives me enough to clean the actual file and start vizualing everything! 